# Algoritmo - Identificação da programação em trecho de interesse 
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Integração dos dados do GTFS com infomações das paradas programadas_

# 1.0 Integração dos dados do formato GTFS
    ''' Mesclagem dos dados de localização das paradas, horarios previstos e viagens programadas em uma unica base '''

In [ ]:
# Importando base com locais previstos de parada
import pandas as pd 
stops_paradas = pd.read_csv('stops_2024', sep= ',')

In [ ]:
# Removendo colunas desnecessárias do df com paradas 
stops_paradas.drop(columns= ['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station', 
                             'stop_timezone', 'wheelchair_boarding'], inplace=True)

In [ ]:
stops_paradas

In [ ]:
# Importando base com horarios previstos de parada  
stop_times = pd.read_csv('stop_times_2024', sep= ',')

# Importando dados com viagens programadas de cada linha 
trips = pd.read_csv('trips_2024', sep=",")

In [ ]:
# Removendo colunas desnecessárias do df com paradas 
stop_times.drop(columns= ['stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'], inplace=True)

In [ ]:
# Removendo informações desnecessárias do df de dados das viagens
trips_tratado = trips.drop(columns=['service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id',
                                    'wheelchair_accessible'])

In [ ]:
stop_times

In [ ]:
# Juunção dos dados das paradas pela mesmo stop_id no df de tempos previstos de passagem
merge_pardas_time = pd.merge(stop_times, stops_paradas, left_on='stop_id', right_on='stop_id')

In [ ]:
merge_pardas_time

In [ ]:
# MESCLANDO BASE COM INFORMAÇÕES DAS PARADAS E TEMPOS DE PASSAGENS COM OS DADOS DAS VIAGENS PROGRAMADAS E AS RESPECTIVAS LINHAS
merge_paradas_time_trips = pd.merge(merge_pardas_time, trips_tratado, left_on='trip_id', right_on='trip_id')

In [ ]:
merge_paradas_time_trips

# 2.0 Filtragem em trecho de interesse 
    '''A partir da base integrada é possivel verificar para uma determinada linha, os locais programados de passagem '''

## 2.1 Identificando paradas programadas 

In [ ]:
# Filtrando no trecho de interesse
''' (A PARTIR DA ESTAÇÃO DO NORTHSHOPING E 120 METROS APOS A ULTIMA ESTAÇÃO) '''
trecho = merge_paradas_time_trips[(merge_paradas_time_trips.stop_lat>=-3.736013)
    & (merge_paradas_time_trips.stop_lon>=-38.566300) & (merge_paradas_time_trips.stop_lon<=-38.542240) 
                                & (merge_paradas_time_trips.stop_lat<=-3.731021)]

In [ ]:
# TODAS AS LINHAS QUE POSSUEM VIAGENS PROGRAMDAS NESSA REGIÃO 
trecho['route_id'].unique()

In [ ]:
# Filtrando linha no trecho de interesse 
linha = trecho[trecho.route_id==28]

In [ ]:
linha

In [ ]:
# Verificando as paradas programadas presentes 
linha['stop_name'].unique()

In [ ]:
# Quantidade de passagens progamadas por paradas 
linha['stop_name'].value_counts()

## 2.1 Visualizando lat lon das paradas por linha

In [ ]:
# Atribuindo valores de lat,lon das paradas a um df
df_geometry = linha.drop_duplicates(subset='stop_name')

In [ ]:
df_geometry

In [ ]:
lat = df_geometry['stop_lat'].mean()
lon = df_geometry['stop_lon'].mean()

In [ ]:
# Criando um mapa centrado em FORTALEZA
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[lat, lon], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in df_geometry.iterrows():
    folium.CircleMarker(location=[row['stop_lat'], row['stop_lon']],
                        radius=10,
                        color='black',
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['stop_name'], ''), max_width=3)).add_to(marker_cluster)

In [ ]:
mapa

# ----------------------------------------------------------------